<a href="https://colab.research.google.com/github/somosnlp/recursos-nlp-es/blob/main/cuantizacion/GPT_J_6B_8bit_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Playing with Spanish 6-Billion GPT-J (8-bit quantized)

> by [@mrm8488](https://twitter.com/mrm8488)

In [ ]:
! nvidia-smi

Tue Aug  9 15:34:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers==4.14.1
!pip install http://35.184.69.80/root/pypi/+f/01a/9628eee5d6858/bitsandbytes-cuda111-0.26.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.9 MB 303 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for bitsandbytes-cuda11.1.1: filename=bitsandbytes_cuda11.1.1-0.26.0-py3-none-any.whl size=3985724 sha256=0e0b41581d09f9048a7010145a9ccaff14449ee220c1f51b7945632e9a460e1e
  Stored in directory: /root/.cache/pip/wheels/93/02/4e/d54aafb3d81e795aae3d1193a5423c3fa563be41ac9f89212e
Successfully built bitsandbytes-cuda11.1.1


In [ ]:
import transformers
import torch

### Converting the model to 8 bits.

We convert EleutherAI's GPT-J-6B model to 8 bits using facebook's [bitsandbytes](https://github.com/facebookresearch/bitsandbytes) library. This reduces the model's size from 20Gb down to just 6Gb.

Note that we don't convert linear layer biases to 8 bit as they take up less that 1% of the model's weight anyway.

In [ ]:
! wget https://huggingface.co/mrm8488/bertin-gpt-j-6B-ES-8bit/resolve/main/Utils.py -O Utils.py

--2022-08-09 15:33:22--  https://huggingface.co/mrm8488/bertin-gpt-j-6B-ES-8bit/resolve/main/Utils.py
Resolving huggingface.co (huggingface.co)... 52.2.34.29, 34.231.117.252, 2600:1f18:147f:e850:d57d:d46a:df34:61ee, ...
Connecting to huggingface.co (huggingface.co)|52.2.34.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5838 (5.7K) [text/plain]
Saving to: ‘Utils.py’

Utils.py            100%[===================>]   5.70K  --.-KB/s    in 0s      

2022-08-09 15:33:22 (80.2 MB/s) - ‘Utils.py’ saved [5838/5838]



In [ ]:
from Utils import GPTJBlock, GPTJForCausalLM

In [ ]:
transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

CKPT = 'mrm8488/bertin-gpt-j-6B-ES-8bit'

tokenizer = transformers.AutoTokenizer.from_pretrained(CKPT)
model = GPTJForCausalLM.from_pretrained(CKPT).to(device)

In [ ]:
import textwrap

In [ ]:
def generate(text: str, max_length: int = 64, do_sample: bool = True, temperature: float = 0.9) -> str:
  prompt = tokenizer(text, return_tensors='pt')
  features = {key: value.to(device) for key, value in prompt.items()}
  out = model.generate(**features, max_length=max_length, do_sample=do_sample, temperature=temperature)
  return tokenizer.decode(out[0])

In [ ]:
offers = [
['¡30% EXTRA! Disfruta de este cupón descuento Venca ¡REBAJAS!',
 '25% MENOS usando el código descuento Yoins al realizar tu compra',
 '10% EXTRA en tus compras al aplicar el cupón descuento Samsung',
 '12% MENOS aplicando este código descuento Sammydress en tu pedido online',
 '-15% al aplicar este código en todos los productos',
 '-20% al introducir el código promocional Mister Spex',
 '-5% aplicando este código promocional MiniInTheBox antes de finalizar tu pedido',
 '-18% introduciendo el cupón descuento Milanoo de Navidad',
 '-10% aplicando este código al comprar 3 artículos en la web',
 '20% MENOS al aplicar el código descuento Miss Hamptons',
 '70% MENOS para hacer calendarios personalizados para hacer calendarios personalizados para hacer calendarios personalizados para hacer calendarios personalizados con este código Vistaprint',
 '10% de ahorro en TODA la web aplicando este código descuento Spartoo ¡Limitado!',
 '20% MENOS al aplicar este código promocional iStock',
 '8% MENOS en tu alojamiento aplicando este cupón Hoteles.com',
 '-15% al usar el código promocional Bikeinn del Cyber Monday',
 '-17% en Yoins introduciendo este código en la web',
 '15% de rebaja en gaming Omen aplicando este código descuento HP',
 '20% código descuento Nutritienda en una selección de marcas',
 '40% MENOS en tus pedidos al aplicar este código promocional Vistaprint',
 '30% usando este código al comprar álbumes de lujo',
 '19% de ahorro al aplicar este código descuento Yoins ¡Celebra el Año Nuevo!',
 '12% código descuento iStock en TODO para nuevos clientes',
 '-10% usando el código descuento Qatar Airways al reservar desde el móvil',
 '-40% usando este código descuento Photobox al comprar un álbum de fotos',
 '-10% aplicando este código ¡Oportunidad Yoins!',
 '5% código descuento Zoombits en Multicompra',
 '10% código descuento Animalear',
 '7% MENOS en tu reserva aplicando este código descuento H10 Hotels',
 '-10% usando este código al comprar muebles en VidaXL',
 '25% MENOS al aplicar este cupón La Tostadora',
 '-8% en electrónica de consumo aplicando este código promocional GearBest',
 '¡-20% aplicando este cupón descuento Lentes-Shop!',
 '-20% en tu moda favorita ¡código Venca!',
 '5% código descuento Agoda para viajar a Shanghai',
 '10% MENOS en snack para tu mascota aplicando este cupón promocional activo',
 '10%. 15% o 20% ¡Elige tu cupón Kiwoko y ahorra en el pienso para tu perro!',
 '¡15% menos con este gran código promocional Runnerinn del Cyber Monday!',
 '5% EXTRA aplicando este código en TODO',
 '20% MENOS usando este código al comprar la mejor moda',
 '30% MENOS en todas las bebidas al aplicar el código promocional Tassimo',
 '10% MENOS al aplicar este código descuento FocalPrice',
 '20% MENOS en una gran variedad de lámparas aplicando este cupón',
 '8% MENOS al aplicar este cupón descuento',
 '12% MENOS al aplicar este cupón descuento Dosfarma',
 '15% MENOS al aplicar el cupón descuento Magix',
 '-5% aplicando este código promocional Spartoo de verano',
 '20% en Novedades Milano con código promocional Fifty Factory',
 '20% MENOS al aplicar el código descuento Zavvi en vinilos',
 '-5% usando este código descuento Zoombits al comprar 2 artículos',
 '-8% aplicando este código',
 '20% adicional en en TODA la web si usas el cupón descuento El Armario de la Tele',
 '5% código descuento en los accesorios de cocina de Zoombits',
 '20% código descuento Lopesan Hoteles']

In [ ]:
text = '''
título: 5% código descuento Zoombits en Multicompra
oferta: 5% en Multicompra

título: -10% usando este cupón descuento Alltricks
oferta: -10%

título: 10% código descuento Animalear
oferta; 10%

título: ¡5% adicional utilizando este código descuento Barceló Hoteles!
oferta: 5% adicional

título: 15% de descuento al utilizar el código promocional Spartoo
oferta: 15%

título: 25% MENOS utilizando este código descuento Groupon
oferta: 25% MENOS

título: -25% en la web usando e cupón descuento vistaprint
oferta: -25% en la web

título: -10% usando este código al comprar muebles en VidaXL
oferta: -10% al comprar muebles

título: -20% en tu moda favorita ¡código Venca!
oferta: -20% en tu moda favorita

título: 19% de ahorro al aplicar este código descuento Yoins ¡Celebra el Año Nuevo!
oferta: 19% de ahorro

título: 8% MENOS en tu alojamiento aplicando este cupón Hoteles.com
oferta: 8% MENOS en tu alojamiento

título::
oferta:'''

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
titles = []
ofertas = []
for offer in tqdm(offers):
  prompt = text.replace('título::', 'título: ' + offer)
  response = generate(prompt, 512, True, temperature = 0.8)
  print(offer)
  print(response.replace(prompt, ''))
  print()
  print("===")
  titles.append(offer)
  ofertas.append(response.replace(prompt, '').split('\n')[0].strip())


  0%|          | 0/49 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


10% MENOS usando el cupón descuento Yoins
 10% MENOS en tus compras
título: 10% de descuento con este cupón descuento vueling
oferta: 10% de descuento
título: 5% MENOS al comprar en Zalando utilizando este cupón descuento Zalando (Nuevo)
of

===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


27% MENOS usando el cupón descuento La Tostadora
 27% MENOS en la compra de tus tostadas
título: 27% MENOS en tu alojamiento
título: -10% con el cupón descuento Groupon
oferta: -10% en tu alojamiento
título: 27% DE AHOR

===


...

-40% utilizando este código promocional Blurb ¡Promoción!
 -40%
título: -20€ de ahorro en tus compras en la web de L’Occitane ¡Promoción!
oferta: -20€ en tus compras
título: -20% en tu segunda compra usando este c

===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


¡15% menos con este gran código promocional Runnerinn del Cyber Monday!
 ¡15% menos con este gran código promocional Runnerinn del Cyber Monday!
título: ¡15% en tu cesta de la compra! ¡código VidaXL!
oferta: ¡15% en tu cesta de la compra!


===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


15% MENOS usando el cupón descuento Tomtop
 15%
título: 10% de descuento en tu pedido si lo haces en Sarenza
oferta: 10% de descuento en tu pedido
título: 30% de descuento en bolsos de Michael Kors
oferta: 30% de descuento
título:

===
5% EXTRA aplicando este código en TODO
 5% EXTRA aplicando este código en TODO
título: -15€ en TODO aplicando este código descuento Groupon
oferta: -15€ en TODO
título: 15% extra en tus compras en Mango
oferta: 15

===


In [ ]:
for t, o in zip(titles, ofertas):
  print('title: ' + t)
  print('offer: ' + o)
  print()

title: 10% MENOS usando el cupón descuento Yoins
offer: 10% MENOS en tus compras

title: 27% MENOS usando el cupón descuento La Tostadora
offer: 27% MENOS en la compra de tus tostadas

title: 10% de ahorro en TODA la web aplicando este código descuento Spartoo ¡Limitado!
offer: 10% de ahorro en TODA la web en las rebajas de invierno

title: 20% MENOS al aplicar este código promocional iStock
offer: 20% MENOS en tu cámara digital

title: 8% MENOS en tu alojamiento aplicando este cupón Hoteles.com
offer: 8% MENOS en tu alojamiento

title: -15% al usar el código promocional Bikeinn del Cyber Monday
offer: -15%

title: -17% en Yoins introduciendo este código en la web
offer: -17% en Yoins

title: 15% de rebaja en gaming Omen aplicando este código descuento HP
offer: 15% de rebaja

title: 20% código descuento Nutritienda en una selección de marcas
offer: 20%

title: 40% MENOS en tus pedidos al aplicar este código promocional Vistaprint
offer: 40% MENOS en tus pedidos

title: 30% usando este

In [ ]:
response = generate(text, 380, True)
print(response)

In [ ]:
text = "¿Cuál es el sentido de la vida?"
response = generate(text, 100, True)
wrapped_lines = textwrap.wrap(response, width=25)
print("\n".join(wrapped_lines))

In [ ]:
text = "¿Cómo se hace una paella?"
response = generate(text, 100, True)
wrapped_lines = textwrap.wrap(response, width=25)
print("\n".join(wrapped_lines))

In [ ]:
text = "¿Qué sexo tiene los ángeles?"
response = generate(text, 100, True)
wrapped_lines = textwrap.wrap(response, width=25)
print("\n".join(wrapped_lines))

In [ ]:
text = "¿Cómo se construyeron las pirámides de Egipto?"
response = generate(text, 266, True)
wrapped_lines = textwrap.wrap(response, width=25)
print("\n".join(wrapped_lines))

In [ ]:
text = "¿Por qué están creando los humanos la inteligencia artificial?"
response = generate(text, 64, True)
wrapped_lines = textwrap.wrap(response, width=25)
print("\n".join(wrapped_lines))

In [ ]:
text = "¿Qué es el amor? El amor es"
response = generate(text, 128, True)
wrapped_lines = textwrap.wrap(response, width=25)
print("\n".join(wrapped_lines))

In [ ]:
text = "¿Quién es Josep Pedrerol? Pedrerol es un periodista deportivo"
response = generate(text, 128, True)
wrapped_lines = textwrap.wrap(response, width=25)
print("\n".join(wrapped_lines))

In [ ]:
text = "Juanse, ¿Por qué discuten las personas? Las personas discuten porque"
response = generate(text, 128, True)
wrapped_lines = textwrap.wrap(response, width=25)
print("\n".join(wrapped_lines))

In [ ]:
text = '''
A continación se describen una serie de países y sus capitales:

España - Madrid
Francia - París
Alemania -'''
response = generate(text)
wrapped_lines = textwrap.wrap(response, width=25)
print("\n".join(wrapped_lines))

In [ ]:
text = '''
Valoraciones de películas con análisis de sentimiento:

Frase: Me gusta mucho
Sentimiento: Positivo

Frase: Fue un horor
Sentimiento: Negativo

Frase: Estuvo muy bien
Sentimiento: Positivo

Frase: Ni fu ni fa
Sentimiento:'''
print(generate(text, max_length=100))

In [ ]:
text = "¿Eres de derechas o de izquierdas?"
print(generate(text))

In [ ]:
text = "¿Cuántos planetas hay en el sistema solar? El sistema sola está compuesto"
print(generate(text))

In [ ]:
text  = '''
Diálogo entre dos amigos:

- Jordi: "Bon día!"
- Adela: Buenos días!
- Jordi:'''
print(generate(text))